## Create SST

In [1]:
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import pandas as pd
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# import netCDF4
# from netCDF4 import *

import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from scipy import interpolate
from scipy.interpolate import griddata
import time
import glob
import dask

from scipy.fftpack import fft
from scipy.fftpack import ifft
import copy
import eofs.standard as Eof_st
from matplotlib.colors import ListedColormap

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import cmocean
import pop_tools
from pop_tools.grid import _compute_corners
import cftime
import geocat.comp
import xesmf as xe


## Useful Functions

In [2]:
#this functions pick data ranges out of an xarray data structure.
def is_dayofyear(dayofyear,dd):
    return (dayofyear==dd)

def is_dayofyear_range(dayofyear,dd1,dd2):
    
    if (dd2 > dd1) & (dd2<366) & (dd1>=1):
        return ((dayofyear>=dd1)&(dayofyear<=dd2))
    elif (dd2 > 366) & (dd1>0):
        dd2 = dd2 - 366
        print('you are wrapping around years... this is a warning if you arent intending to')
        return ((dayofyear>=dd1)|(dayofyear<=dd2))
    elif dd1 < 1:
        print('you are wrapping around years... this is a warning if you arent intending to')
        dd1 = dd1+366
        return ((dayofyear>=dd1)|(dayofyear<=dd2))
    else:
        print('this is an edge case you did not foresee')
        return np.nan
    
def cal_ano_dcli(var):
    '''return [var_ano, var_dcli], Taxis= axis of time'''
    var_dcli=var.groupby('time.dayofyear').mean(dim='time')
    var_ano= var - var.mean(dim = 'time')
    
    return var_ano, var_dcli




#make MJO base line months
def is_NDJFM(dayofyear):
    return ((dayofyear==11)|(dayofyear==12)|(dayofyear==1)|(dayofyear==2)|(dayofyear==3))


    
def is_doyrange(doy,dd,tod,hh):
    daywind = 15
    if (dd - daywind) < 1:
        # print('in 1')
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        # print('in 2')
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        # print('in 3')
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)
    
def gen_corner_calc(ds, cell_corner_lat='ULAT', cell_corner_lon='ULONG'):
    """
    Generates corner information and creates single dataset with output
    """

    cell_corner_lat = ds[cell_corner_lat]
    cell_corner_lon = ds[cell_corner_lon]
    # Use the function in pop-tools to get the grid corner information
    corn_lat, corn_lon = _compute_corners(cell_corner_lat, cell_corner_lon)

    # Make sure this returns four corner points
    assert corn_lon.shape[-1] == 4

    lon_shape, lat_shape = corn_lon[:, :, 0].shape
    out_shape = (lon_shape + 1, lat_shape + 1)

    # Generate numpy arrays to store destination lats/lons
    out_lons = np.zeros(out_shape)
    out_lats = np.zeros(out_shape)

    # Assign the northeast corner information
    out_lons[1:, 1:] = corn_lon[:, :, 0]
    out_lats[1:, 1:] = corn_lat[:, :, 0]

    # Assign the northwest corner information
    out_lons[1:, :-1] = corn_lon[:, :, 1]
    out_lats[1:, :-1] = corn_lat[:, :, 1]

    # Assign the southwest corner information
    out_lons[:-1, :-1] = corn_lon[:, :, 2]
    out_lats[:-1, :-1] = corn_lat[:, :, 2]

    # Assign the southeast corner information
    out_lons[:-1, 1:] = corn_lon[:, :, 3]
    out_lats[:-1, 1:] = corn_lat[:, :, 3]

    return out_lats, out_lons
# def is_NDJFM(dayofyear):
#     return ((dayofyear==12)|(dayofyear==1)|(dayofyear==2))

# def is_NDJFM(dayofyear):
#     return ((dayofyear==1)|(dayofyear==2)|(dayofyear==3)|(dayofyear==4)|(dayofyear==5)|(dayofyear==6)|(dayofyear==7)|(dayofyear==8)|(dayofyear==9)|(dayofyear==10)|(dayofyear==11)|(dayofyear==12))

#  NDJFM_data = GPH_E20c.sel(time=is_NDJFM(GPH_E20c['time.month']))

## First Load the CESM LENS Data
need Z500


In [3]:
# #!!!!!uncomment this next line to do the whole dataset (there is another line to comment out below):!!!!

# DSz500 = xr.open_mfdataset(['/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/daily/Z500/b.e11.B1850C5CN.f09_g16.005.cam.h1.Z500.05000101-05991231.nc',
#                          '/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/daily/Z500/b.e11.B1850C5CN.f09_g16.005.cam.h1.Z500.06000101-06991231.nc',
#                          '/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/daily/Z500/b.e11.B1850C5CN.f09_g16.005.cam.h1.Z500.07000101-07991231.nc'],combine='by_coords')

# #!!!!!!uncomment the previous line to do the whole dataset (there is another line to comment out below):!!!

#AND! comment this next line to do the whole dataset: 




DSz500 = xr.open_dataset('/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/daily/Z500/b.e11.B1850C5CN.f09_g16.005.cam.h1.Z500.05000101-05991231.nc')
DSz500=DSz500.drop(['slon','slat'])
#AND! comment the previous line to do the whole dataset: 

DSz500=DSz500.convert_calendar("standard", use_cftime=True)
DSz500=DSz500.load()

## Regrid and Save new as lat/lon files... 
## !!!! ONLY RUN THIS IF THE FILES DO NOT EXIST ALREADY IN THE PATH !!!
- it takes forever

In [7]:
%%time
path = '/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/'
svpath = '/glade/work/wchapman/For_Tony/'
sst_nc_1=path+'b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01010102-01510101.nc'
sst_nc_2=path+'b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01510102-02010101.nc'
sst_nc_3=path+'b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02010102-02510101.nc'
sst_nc_4=path+'b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02510102-03010101.nc'
sst_nc_5=path+'b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03010102-03510101.nc'
sst_nc_6=path+'b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03510102-04010101.nc'

sst_nc_all = [sst_nc_1,sst_nc_2,sst_nc_3,sst_nc_4,sst_nc_5,sst_nc_6]

for ncsst in sst_nc_all:
    print(ncsst.split('/')[-1])
    DSsst = xr.open_dataset(ncsst)
    lat_corners, lon_corners = gen_corner_calc(DSsst)

    DSsst = DSsst.rename({'TLAT': 'lat', 'TLONG': 'lon'}).drop(
        ['ULAT', 'ULONG']
    )

    DSsst['lon_b'] = (('nlat_b', 'nlon_b'), lon_corners)
    DSsst['lat_b'] = (('nlat_b', 'nlon_b'), lat_corners)

    regrid = xe.Regridder(DSsst,DSz500, method='conservative', weights='sst_to_grid_woa_tx0.1v3_conservative.nc')
    regridded_model_full = regrid(DSsst)
    
    regridded_model_full = regridded_model_full.drop(['ANGLE','ANGLET','DXT','DXU','DYT','DYU','HT','HTE','HTN','HU','HUS','HUW','KMT','KMU'])
    
    #save out: 
    svname = svpath+ncsst.split('/')[-1]
    regridded_model_full.to_netcdf(svname)

b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01010102-01510101.nc


/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01510102-02010101.nc


/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02010102-02510101.nc


...

/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03010102-03510101.nc


/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03510102-04010101.nc


/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


# Start Here

In [3]:
# #!!!!!uncomment this next line to do the whole dataset (there is another line to comment out below):!!!!
DSsst = xr.open_mfdataset(['/glade/work/wchapman/For_Tony/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01010102-01510101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01510102-02010101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02010102-02510101.nc',
                         '/glade/work/wchapman/For_Tony/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02510102-03010101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03010102-03510101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03510102-04010101.nc',
                          ],combine='by_coords',parallel=True)

# #!!!!!!uncomment the previous line to do the whole dataset (there is another line to comment out below):!!!

#AND! comment this next line to do the whole dataset: 
# DSsst = xr.open_dataset('/glade/work/wchapman/For_Tony/b.e11.B1850C5CN.f09_g16.005.pop.h.nday1.SST.05000101-05991231_V2.nc')


#AND! comment the previous line to do the whole dataset: 
print('...convert calendar...')
DSsst=DSsst.convert_calendar("standard", use_cftime=True)
print('...loading...')
# Specify the desired chunk sizes for each dimension
chunk_sizes = {'lat': 192, 'lon': 288, 'time': 100}  # -1 means keep the existing chunk size
# Rechunk the dataset
DSsst = DSsst.chunk(chunk_sizes)
# DSsst=DSsst.load()
DSsst=DSsst.load()

## Make Climatology
- This takes about 1000 years.. it's better to just load the files if they have already been made

In [3]:
DSsst = xr.open_dataset('/glade/work/wchapman/For_Tony//SST_camgrid_gathered_Areas.nc')
DSsst

<xarray.Dataset>
Dimensions:      (time: 109500, lat: 192, lon: 288)
Coordinates:
  * time         (time) object 0101-01-02 00:00:00 ... 0401-01-01 00:00:00
  * lat          (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon          (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    SST          (time, lat, lon) float32 ...
    REGION_MASK  (lat, lon) float64 ...
    UAREA        (lat, lon) float64 ...
    TAREA        (lat, lon) float64 ...

In [2]:
%%time
SST_climo = xr.zeros_like(DSsst['SST'])
print('... creating climo from centered 30 day average ...')
for ee,dayhr in (enumerate(DSsst.time)):
    if ee%10 ==0:
        print('doing ',ee,' of 365')
    dooDOY = dayhr['time.dayofyear']
    hh=dayhr['time.hour']
    Dtemp = DSsst.sel(time=is_doyrange(DSsst['time.dayofyear'],dooDOY,DSsst['time.hour'],hh))[['SST']].mean(['time'])
    SST_climo[ee,:,:] = Dtemp['SST'].values
    
    if ee == (365)+8:
        endee=ee
        enddate = dayhr
        break
print("this is much faster but only works with noleap calendar:")  
SSTnp = np.array(SST_climo[:365,:,:])
arrays_to_concat = [SSTnp] * 300
concatenated_arr = np.concatenate(arrays_to_concat, axis=0)
SST_climo[:,:,:]=concatenated_arr
dr = pd.date_range(start='1801-01-02', end='2101-01-01', freq='D')
dates = dr[(dr.day != 29) | (dr.month != 2)]
DSsst['SST_climo'] = SST_climo
DSsst['time']=dates
svname = '/glade/scratch/wchapman/data_for_KJM//'+'SST_CESM2_100_400_repeats.nc'
DSsst.to_netcdf(svname)
print('otherwise you have to do this: (uncomment below)')

# print('...now repeating climo...') 
# for ee,dayhr in (enumerate(DSsst.time)):
#     dm = dayhr['time.month'].values
#     dd = dayhr['time.day'].values
#     dy = DSsst['time.year'][0].values

#     getday = f'{dy:04}'+'-'+f'{dm:02}'+'-'+f'{dd:02}'
    
#     #leap year shenanigans:
#     if f'{dm:02}'+'-'+f'{dd:02}' == '01-01': 
#         getday = f'{dy:04}'+'-01-02'
        
#     if ee%500 ==0:
#         print('doing ',ee,' of ', len(DSsst['time']))
#     try:
#         DtempSST = SST_climo.sel(time=getday).squeeze()
#     except: 
#         DtempSST = SST_climo.sel(time=getday,method='nearest').squeeze()
#     SST_climo[ee,:,:] = DtempSST.values
    
# DSsst['SST_climo'] = SST_climo

# svname = '/glade/work/wchapman/For_Tony/'+'SST_CESM2_100_400.nc'
# DSsst.to_netcdf(svname)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


## For Tony to do.. Create your anomaly: 
- subtract Z500_climo from Z500 and save a new variable Z500a
- Take and average over the north Pacific (lat[20,70],lon[150-240])
- - This will make the "index" of interest